In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
mobile = pd.read_csv('/kaggle/input/mobile-price-classification/train.csv')
mobile.head(10)

In [ ]:
mobile.info()

In [ ]:
mobile["price_range"].value_counts()

In [ ]:
mobile.describe()

In [ ]:
import matplotlib.pyplot as plt
mobile.hist(bins=50, figsize=(20,15))
plt.show

In [ ]:
from sklearn.model_selection import train_test_split
train_set, test_set = train_test_split(mobile, test_size=0.2, random_state=42)
print(len(train_set))
print(len(test_set))

In [ ]:
corr_matrix = mobile.corr()
corr_matrix["price_range"].sort_values(ascending=False)

In [ ]:
from pandas.plotting import scatter_matrix
attributes = ["price_range", "ram", "battery_power", "px_width", "px_height", "touch_screen"]
scatter_matrix(mobile[attributes], figsize=(12,8))

In [ ]:
mobile["px_res"] = mobile["px_width"] * mobile["px_height"]

In [ ]:
corr_matrix = mobile.corr()
corr_matrix["price_range"].sort_values(ascending=False)

In [ ]:
mobile = train_set.drop("price_range", axis=1)
mobile_labels = train_set["price_range"].copy()

In [ ]:
from sklearn.tree import DecisionTreeRegressor

tree_reg = DecisionTreeRegressor()
tree_reg.fit(mobile,mobile_labels)

In [ ]:
from sklearn.metrics import mean_squared_error

mobile_predictions = tree_reg.predict(mobile)
tree_mse = mean_squared_error(mobile_labels, mobile_predictions)
tree_rmse = np.sqrt(tree_mse)
print(tree_rmse)

In [ ]:
from sklearn.model_selection import cross_val_score

scores = cross_val_score(tree_reg, mobile, mobile_labels, scoring="neg_mean_squared_error", cv=10)
tree_rmse_scores = np.sqrt(-scores)

In [ ]:
print("Scores:", tree_rmse_scores)
print("Mean:", tree_rmse_scores.mean())

In [ ]:
mobile_test = train_set.drop("price_range", axis=1)
mobile_test_labels = train_set["price_range"].copy()

mobile_test_predictions = tree_reg.predict(mobile_test)

In [ ]:
print("Predictions: \t",list(int(i) for i in mobile_test_predictions[70:90]))
print("Actual: \t",list(mobile_test_labels[70:90]))

In [ ]:
test_scores = cross_val_score(tree_reg, mobile_test, mobile_test_labels, scoring="neg_mean_squared_error", cv=10)
test_tree_rmse_scores = np.sqrt(-test_scores)

In [ ]:
print("Test Scores:", test_tree_rmse_scores)
print("Test Mean:", test_tree_rmse_scores.mean())